### For off-target deletion

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

figsuplix = 'pdf'

In [2]:
def is_Exist_file(path):
    import os
    if os.path.exists(path):
        os.remove(path)


def mkdir(path):
    import os
    path = path.strip()  # 去除首位空格
    path = path.rstrip("\\")  # 去除尾部 \ 符号
    isExists = os.path.exists(path)  # 判断路径是否存在
    # 判断结果
    if not isExists:
        os.makedirs(path)  # 如果不存在则创建目录
        print(path + ' 创建成功')
    else:
        print(path + ' 目录已存在')  # 如果目录存在则不创建，并提示目录已存在

In [3]:
## mutation 坐标转换
#####################################################################################
## 反向互补
def reverseComplement(seq):
    """
     生成反向互补序列
     :param seq:
     :return:revComSeq
     """
    ATCG_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'a': 't', 't': 'a', 'c': 'g', 'g': 'c', 'N': 'N'}
    revComSeq = ''
    for i in seq:
        revComSeq = ATCG_dict[i] + revComSeq
    return revComSeq


## 转换核心函数
def transformer_core(mut, index_dict):
    if 'M' in mut:
        inf, sup = int(mut.split(':')[0][1:]), int(mut.split(':')[1][:3])
        bn, an = mut.split('_')[1].split(':')[0], mut.split('_')[1].split(':')[1]
        new_inf, new_sup = index_dict[inf], index_dict[sup]
        new_bn, new_an = reverseComplement(bn), reverseComplement(an)
        return '%s%s:%sM_%s:%s'%(mut[0], new_inf, new_sup, new_bn, new_an)
    elif 'I' in mut:
        new_inser_site = index_dict[int(mut.split('_')[0][1:-1])]
        new_inser_nucles = reverseComplement(mut.split('_')[-1])
        return '%s%sI_%s'%(mut[0], new_inser_site, new_inser_nucles)
    elif 'D' in mut:
        inf, sup = int(mut.split(':')[0][1:]), int(mut.split(':')[-1][:-1])
        new_inf, new_sup = index_dict[inf], index_dict[sup]
        return '%s%s:%sD'%(mut[0], new_inf, new_sup)
    else:
        return mut


## 解析 all_mutation & target_mutation 信息
def parse_mutation_information(mutation):
    '''
    mutation = 'X0+Y179:180D.Y186:186M_C:A+Z196I_T'
    # mutation = 'X0+Y0+Z0'
    muts = parse_mutation_information(mutation)
    '''
    import re
    mut_list = re.split('\+|\.', mutation)
    mut_dict = {'X': [], 'Y': [], 'Z': []}
    for mut in mut_list:
        mut_dict[mut[0]].append(mut)
    return mut_dict


## 调整突变坐标的参考系：以 gRNASeq 为基
def adjust_off_target_mutaion_reference_system(mutation):
    '''
    mutation = 'X164I_T+Y178I_C+Z0'
    mutation = 'X164:164D+Y187I_G+Z199:199M_C:T'
    mutation = 'X0+Y0+Z0'
    adjust_off_target_mutaion_reference_system(mutation)
    '''
    ## 长序列与短序列对应字典
    index_dict = {}
    for i in range(20):
        index_dict[125 + i] = -19 + i
    for i in range(63):
        index_dict[145 + i] = 1 + i
    for i in range(20):
        index_dict[208 + i] = 64 + i
    ## 解析 all_mutation & target_mutation 信息
    mut_dict = parse_mutation_information(mutation)
    new_mut_dict = {'X': '', 'Y': '', 'Z': ''}
    for label, mut_list in mut_dict.items():
        a_list = []
        for mut in mut_list:
            new_mut = transformer_core(mut, index_dict)
            a_list.append(new_mut)
        a_list.sort(reverse=False)
        new_mut_dict[label] = '.'.join(a_list)
    new_list = []
    for label in ['X', 'Y', 'Z']:
        new_list.append(new_mut_dict[label])
    return '+'.join(new_list)
#####################################################################################

In [4]:
## up, core, down 区域内突变核苷酸总长度
def region_mutation_length(mutation):
    mut_list = mutation.split(".")
    mut_length = 0
    for mut in mut_list:
        if 'M' in mut:
            mut_length += len(mut.split(":")[-1])
        elif 'I' in mut:
            mut_length += len(mut.split("_")[-1])
        elif 'D' in mut:
            inf, sup = int(mut.split(':')[0][1:]), int(mut.split(':')[-1][:-1])
            delt_length = sup - inf + 1
            mut_length += delt_length
        else:
            pass
    return mut_length


def mutation_region_splitting(data):
    ## step 1: mutaion 区域划分
    data['new_mutation'] = data['all_mutation'].apply(lambda x: adjust_off_target_mutaion_reference_system(x))
    ## up, core, down 区域内突变类型
    data['up_mutation'] = data['new_mutation'].apply(lambda x: x.split('+')[0])
    data['core_mutation'] = data['new_mutation'].apply(lambda x: x.split('+')[1])
    data['down_mutation'] = data['new_mutation'].apply(lambda x: x.split('+')[2])
    ## up, core, down 区域内间断突变的个数
    data['up_mut_num'] = data['up_mutation'].apply(lambda x: len(x.split('.')) if x != 'X0' else 0)
    data['core_mut_num'] = data['core_mutation'].apply(lambda x: len(x.split('.')) if x != 'Y0' else 0)
    data['down_mut_num'] = data['down_mutation'].apply(lambda x: len(x.split('.')) if x != 'Z0' else 0)
    ## up, core, down 区域内突变核苷酸总长度
    data['up_mut_length'] = data['up_mutation'].apply(lambda x: region_mutation_length(x))
    data['core_mut_length'] = data['core_mutation'].apply(lambda x: region_mutation_length(x))
    data['down_mut_length'] = data['down_mutation'].apply(lambda x: region_mutation_length(x))
    ############################################################
    print("\nFirst: 统计 X0+Y0+Z0、XA+Y0+Z0、X0+YA+Z0、X0+Y0+ZA、XA+YA+Z0、XA+Y0+ZA、X0+YA+ZA、XA+YA+ZA 的类型占比")
    data_000 = data.loc[(data['up_mut_num']==0) & (data['core_mut_num']==0) & (data['down_mut_num']==0), :]
    data_100 = data.loc[(data['up_mut_num']!=0) & (data['core_mut_num']==0) & (data['down_mut_num']==0), :]
    data_010 = data.loc[(data['up_mut_num']==0) & (data['core_mut_num']!=0) & (data['down_mut_num']==0), :]
    data_001 = data.loc[(data['up_mut_num']==0) & (data['core_mut_num']==0) & (data['down_mut_num']!=0), :]
    data_110 = data.loc[(data['up_mut_num']!=0) & (data['core_mut_num']!=0) & (data['down_mut_num']==0), :]
    data_101 = data.loc[(data['up_mut_num']!=0) & (data['core_mut_num']==0) & (data['down_mut_num']!=0), :]
    data_011 = data.loc[(data['up_mut_num']==0) & (data['core_mut_num']!=0) & (data['down_mut_num']!=0), :]
    data_111 = data.loc[(data['up_mut_num']!=0) & (data['core_mut_num']!=0) & (data['down_mut_num']!=0), :]
    ## count
    count_dict = {}
    count_000 = data_000.shape[0]
    count_100, count_010, count_001 = data_100.shape[0], data_010.shape[0], data_001.shape[0]
    count_110, count_101, count_011 = data_110.shape[0], data_101.shape[0], data_011.shape[0]
    count_111 = data_111.shape[0]
    count_dict['X0+Y0+Z0'] = count_000
    count_dict['XA+Y0+Z0'] = count_100
    count_dict['X0+YA+Z0'] = count_010
    count_dict['X0+Y0+ZA'] = count_001
    count_dict['XA+YA+Z0'] = count_110
    count_dict['XA+Y0+ZA'] = count_101
    count_dict['X0+YA+ZA'] = count_011
    count_dict['XA+YA+ZA'] = count_111
    print("X0+Y0+Z0:", count_000)
    print("XA+Y0+Z0:", count_100)
    print("X0+YA+Z0:", count_010)
    print("X0+Y0+ZA:", count_001)
    print("XA+YA+Z0:", count_110)
    print("XA+Y0+ZA:", count_101)
    print("X0+YA+ZA:", count_011)
    print("XA+YA+ZA:", count_111)
    count_sum = data.shape[0]
    print("Distribution of count ratio:", round(count_000/count_sum, 3), 
                        round(count_100/count_sum, 3), round(count_010/count_sum, 3), round(count_001/count_sum, 3), 
                        round(count_110/count_sum, 3), round(count_101/count_sum, 3), round(count_011/count_sum, 3), 
                        round(count_111/count_sum, 3))
    return (data_000, data_100, data_010, data_001, data_110, data_101, data_011, data_111, count_dict)

###### Get deletion off-target data & on-target data

In [5]:
## get deletion off-target data & on-target data
def get_deletion_and_on_target_data(data_dir, mut_type, reads_num, barcode_num):
    delt_path = data_dir + '/off-target.%s.accurate.offseq'%(mut_type)
    delt = pd.read_csv(delt_path, sep='\t')
    delt = delt.loc[(delt['reads_num']>=reads_num) & (delt['barcode_num']>=barcode_num), :]
    delt.reset_index(drop=True, inplace=True)
    print(delt.shape)
    ## data 根据 up, core, down 区域突变与否划分数据集
    delt_000, delt_100, delt_010, delt_001, delt_110, delt_101, delt_011, delt_111, count_dict = mutation_region_splitting(delt)
    return (delt, delt_000, delt_100, delt_010, delt_001, delt_110, delt_101, delt_011, delt_111)


## get on-target data
def get_on_target_data(data_dir, reads_num, barcode_num):
    ######################################
    ## perfect
    data_path = data_dir + '/off-target.perfect.accurate.offseq'
    ## read
    data = pd.read_csv(data_path, sep='\t')
    data = data.loc[(data['reads_num']>=reads_num) & (data['barcode_num']>=barcode_num), :]
    data.reset_index(drop=True, inplace=True)
    print(data.shape)
    ## data 根据 up, core, down 区域突变与否划分数据集
    data_000, data_100, data_010, data_001, data_110, data_101, data_011, data_111, count_dict = mutation_region_splitting(data)
    return (data, data_000, data_100, data_010, data_001, data_110, data_101, data_011, data_111)


## for matching off-target deletion with on-target data 
def Obtain_deletion_data(delt, data, delt_010, delt_110, delt_111, delt_011, 
                         data_000, data_100, data_101, data_001, save_dir): 
    ## 确定 deletion id
    ## 1、deletion X0+YA+Z0
    print("\n1、deletion X0+YA+Z0 ")
    delt_id_1 = delt_010[['sgRNA_name', 'up_mutation', 'down_mutation']].drop_duplicates()
    on_id_1 = data_000[['sgRNA_name', 'up_mutation', 'down_mutation']].drop_duplicates()
    comm_id_1 = pd.merge(delt_id_1, on_id_1, how='inner', on=['sgRNA_name', 'up_mutation', 'down_mutation'])
    print("X0+YA+Z0 deletion id:", delt_id_1.shape)
    print("X0+Y0+Z0 perfect id:", on_id_1.shape)
    print("common X0+YA+Z0 & perfect id", comm_id_1.shape)

    ## 2、deletion XA+YA+Z0
    print("\n2、deletion XA+YA+Z0 ")
    delt_id_2 = delt_110[['sgRNA_name', 'up_mutation', 'down_mutation']].drop_duplicates()
    on_id_2 = data_100[['sgRNA_name', 'up_mutation', 'down_mutation']].drop_duplicates()
    comm_id_2 = pd.merge(delt_id_2, on_id_2, how='inner', on=['sgRNA_name', 'up_mutation', 'down_mutation'])
    print("XA+YA+Z0 deletion id:", delt_id_2.shape)
    print("XA+Y0+Z0 perfect id:", on_id_2.shape)
    print("common XA+YA+Z0 & perfect id:", comm_id_2.shape)

    ## 3、deletion XA+YA+ZA
    print("\n3、insertion XA+YA+ZA")
    delt_id_3 = delt_111[['sgRNA_name', 'up_mutation', 'down_mutation']].drop_duplicates()
    on_id_3 = data_101[['sgRNA_name', 'up_mutation', 'down_mutation']].drop_duplicates()
    comm_id_3 = pd.merge(delt_id_3, on_id_3, how='inner', on=['sgRNA_name', 'up_mutation', 'down_mutation'])
    print("XA+YA+ZA deletion id:", delt_id_3.shape)
    print("XA+Y0+ZA perfect id:", on_id_3.shape)
    print("common XA+YA+ZA & perfect id:", comm_id_3.shape)

    ## 4、deletion X0+YA+ZA
    print("\n4、deletion X0+YA+ZA ")
    delt_id_4 = delt_011[['sgRNA_name', 'up_mutation', 'down_mutation']].drop_duplicates()
    on_id_4 = data_001[['sgRNA_name', 'up_mutation', 'down_mutation']].drop_duplicates()
    comm_id_4 = pd.merge(delt_id_4, on_id_4, how='inner', on=['sgRNA_name', 'up_mutation', 'down_mutation'])
    print("X0+YA+ZA deletion id:", delt_id_4.shape)
    print("X0+Y0+ZA perfect id:", on_id_4.shape)
    print("common X0+YA+ZA & perfect id:", comm_id_4.shape)

    ## merging 
    print("\nmerging")
    comm_id = pd.concat([comm_id_1, comm_id_2, comm_id_3, comm_id_4], axis=0)
    ## For perfect
    on_id = pd.merge(data[['sgRNA_name', 'up_mutation', 'down_mutation', 'reads_num', 'barcode_num', 'off-target_eff']], 
                     comm_id, how='inner', on=['sgRNA_name', 'up_mutation', 'down_mutation'])
    on_id.rename(columns={'reads_num': 'p_reads_num', 'barcode_num': 'p_barcode_num', 'off-target_eff': 'on-target_eff'}, 
                 inplace=True)
    print("perfect on_id:", on_id.shape)
    delt_id = pd.merge(delt, on_id, how='inner', on=['sgRNA_name', 'up_mutation', 'down_mutation'])
    ## 可用于分析的
    delt1 = delt_id.loc[(delt_id['up_mut_num']==0) & (delt_id['core_mut_num']!=0) & (delt_id['down_mut_num']==0), :]
    delt2 = delt_id.loc[(delt_id['up_mut_num']!=0) & (delt_id['core_mut_num']!=0) & (delt_id['down_mut_num']==0), :]
    delt3 = delt_id.loc[(delt_id['up_mut_num']!=0) & (delt_id['core_mut_num']!=0) & (delt_id['down_mut_num']!=0), :]
    delt4 = delt_id.loc[(delt_id['up_mut_num']==0) & (delt_id['core_mut_num']!=0) & (delt_id['down_mut_num']!=0), :]
    print("\nSum number - delt_id.shape:", delt_id.shape)
    print("X0+YA+Z0 delt1.shape:", delt1.shape)
    print("XA+YA+Z0 delt2.shape:", delt2.shape)
    print("XA+YA+ZA delt3.shape:", delt3.shape)
    print("X0+YA+ZA delt4.shape:", delt4.shape)
    ## to save
    mkdir(save_dir)
    delt_id.to_excel(save_dir + '/off-target deletion data.xlsx', index=False)
    return (delt_id, delt1, delt2, delt3, delt4)

#### The distribution of deletion position

In [6]:
## deletion position
def deletion_position(mutation):
    import re
    mut_list = re.split('\+|\.', mutation)
    positions = []
    for mut in mut_list:
        if "D" in mut:
            inf , sup = int(mut.split(":")[0][1:]), int(mut.split(":")[1][:-1])
            for pos in range(inf, sup+1, 1):
                positions.append(pos)
        else:
            pass
    return str(positions)


### 合并所有的 delt
def obtain_deletion_position_distribution(delt1, delt2, delt3, delt4, save_dir):
    ## concat
    delt_s = pd.concat([delt1, delt2, delt3, delt4], axis=0)
    delt11 = delt_s.loc[(delt_s['core_mut_length']==1) & (delt_s['on-target_eff']>=0.05), :]
    print(delt_s.shape, delt11.shape, len(list(delt11['sgRNA_name'].unique())))
    delt11['position'] = delt11['core_mutation'].apply(lambda x: deletion_position(x))
    delt11['position'] = delt11['position'].apply(lambda x: eval(x)[0])
    delt11['reltv_eff'] = delt11.apply(lambda row: row['off-target_eff']/row['on-target_eff'], axis=1)
    stat_eff_df = delt11[['position', 'off-target_eff', 'on-target_eff', 'reltv_eff']]
    ## to save
    stat_eff_df.to_excel(save_dir + '/deletion_distribution_analysis.xlsx', index=False)
    ## deletion position distribution 
    stat_eff_df['count'] = 1
    stat_df = stat_eff_df[['position', 'count']].groupby('position').sum()
    stat_df.sort_values(by='position', ascending=True, inplace=True)
    stat_df.reset_index(drop=False, inplace=True)
    return stat_df, stat_eff_df
#####################################################################################

In [7]:
## 柱状图
def get_xticks(stat_df):
    ## labels 
    pam_dict = {41: 'N', 42: 'G', 43: 'G'}
    pos_list = stat_df['position'].tolist()
    xticks = []
    for pos in pos_list:
        if pos not in pam_dict:
            xticks.append(pos-20)
        else:
            xticks.append(pam_dict[pos])
    return xticks


def plot_single_bar(data_list, xticks, title, save_dir, 
                    xlabel='Target+PAM Insertion Position', ylabel='Count'):
    import matplotlib.pyplot as plt
    # 设置默认绘图风格
    plt.style.use("seaborn-white")  
    fig, ax = plt.subplots(1,1, figsize=(12, 4))

    plt.bar(range(len(data_list)), data_list, color='lightseagreen')
    ## 坐标轴不可见
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ## xlabel, ylabel
    plt.xlabel(xlabel, fontsize=12, weight='bold')
    plt.ylabel(ylabel, fontsize=12, weight='bold')
    ## xticks
    plt.xticks(range(len(xticks)), xticks, fontsize=12, weight='bold')
    ## title
    plt.title(title, fontsize=12, weight='bold')
    ## save
    mkdir(save_dir)
    savefig_path = save_dir + '/%s.%s'%(title, figsuplix)
    plt.savefig(savefig_path, dpi=300, bbox_inches='tight')
    plt.show()


## plot effect of deletion position on efficiency
def plot_deletion_reltv_eff(stat_eff_df, save_dir):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # 设置默认绘图风格
    plt.style.use("seaborn-white")  
    fig, ax = plt.subplots(1,1, figsize=(6, 3))
#     fig, ax = plt.subplots(1,1, figsize=(12, 6))
    
    order = [(i) for i in range(21, 43)]
    ax = sns.boxplot(x='position', y='reltv_eff', data=stat_eff_df, width=0.4, color='white', 
                     fliersize=0.5, linewidth=0.5, order=order)
    ## 坐标轴不可见
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax = sns.stripplot(x='position', y='reltv_eff', data=stat_eff_df, 
                       color="orange", jitter=0.1, size=0.8, order=order)
    ## xlabel, ylabel
    plt.xlabel("")
    plt.ylabel("")
    ## xticks
    xticks = list(range(1, 21, 1)) + ['N', 'G']
    plt.xticks(range(len(xticks)), xticks, fontsize=6, weight='bold')
    plt.yticks(fontsize=6, weight='bold')
    ## ylim
    plt.xlim(-1, 22)
    title = 'Effect of off-target deletion position on gRNA acitvity - D-1'
    mkdir(save_dir)
    savefig_path = save_dir + '/%s.%s'%(title, figsuplix)
    plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')
    plt.show()
    
    
def main_plot_off_target_deletion_effect(delt1, delt2, delt3, delt4, save_dir):
    ## plot the distribution of deletion position
    stat_df, stat_eff_df = obtain_deletion_position_distribution(delt1, delt2, delt3, delt4, save_dir)
    xticks = get_xticks(stat_df)
    data_list = stat_df['count'].tolist()
    title = "position distribution of off-target deletion - D-1"
    ylabel = 'off-target id count'
    xlabel = 'Target+PAM Deletion Position'
    plot_single_bar(data_list, xticks, title, save_dir, xlabel, ylabel)
    ## plot distribution of relative efficiency of deletion
    plot_deletion_reltv_eff(stat_eff_df, save_dir)